We have a number of decent tutorials on the keras blog, as well as Challots book and notebooks:
* https://blog.keras.io/keras-as-a-simplified-interface-to-tensorflow-tutorial.html
* https://blog.keras.io/building-powerful-image-classification-models-using-very-little-data.html
* https://github.com/fchollet/deep-learning-with-python-notebooks

Lets reimplement [5.3-using-a-pretrained-convnet.ipynb](https://github.com/fchollet/deep-learning-with-python-notebooks/blob/master/5.3-using-a-pretrained-convnet.ipynb) starting with a binary classifier using a subset of the image data. 

In [3]:
import os
import numpy as np
import keras
from keras.preprocessing.image import ImageDataGenerator
keras.__version__

'2.1.3'

In [2]:
from keras.applications import VGG16

conv_base = VGG16(weights='imagenet',
                  include_top=False,
                  input_shape=(150, 150, 3))

In [5]:
base_dir = '/Users/robincole/Documents/Data/PatternNet/subset_images'

train_dir = os.path.join(base_dir, 'train')
validation_dir = os.path.join(base_dir, 'validation')
test_dir = os.path.join(base_dir, 'test')

datagen = ImageDataGenerator(rescale=1./255)
batch_size = 20

def extract_features(directory, sample_count):
    features = np.zeros(shape=(sample_count, 4, 4, 512))
    labels = np.zeros(shape=(sample_count))
    generator = datagen.flow_from_directory(
        directory,
        target_size=(150, 150),
        batch_size=batch_size,
        class_mode='binary')
    i = 0
    for inputs_batch, labels_batch in generator:
        features_batch = conv_base.predict(inputs_batch)
        features[i * batch_size : (i + 1) * batch_size] = features_batch
        labels[i * batch_size : (i + 1) * batch_size] = labels_batch
        i += 1
        if i * batch_size >= sample_count:
            # Note that since generators yield data indefinitely in a loop,
            # we must `break` after every image has been seen once.
            break
    return features, labels

In [6]:
%%time
train_features, train_labels = extract_features(train_dir, 2000)
validation_features, validation_labels = extract_features(validation_dir, 1000)
test_features, test_labels = extract_features(test_dir, 1000)

Found 1200 images belonging to 2 classes.


FileNotFoundError: [Errno 2] No such file or directory: '/Users/robincole/Documents/Data/PatternNet/subset_images/validation'

Need to get data organised in the right format. To do - get familiar with keras api